In [1]:
import numpy as np
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
import matplotlib.pyplot as plt
import pickle
import scipy.stats
import itertools

In [2]:
import traceback

In [3]:
from bci_plot.gen_data.robot import process_logs
from bci_plot.gen_data.robot import resample_logs
from bci_plot.metadata import robot_random

In [4]:
import pathlib
head = pathlib.Path('/data/raspy/robot')

In [5]:
subjects = ['S2', 'H1', 'H2', 'H4']

In [6]:
def get_total_movement_time(result, threshold=0.2):
    try:
        idxs = np.nonzero(result['movement_cum_dt'] == 0)[0]
        idxs = idxs[idxs > 0]
        out = result['movement_cum_dt'][idxs-1].sum()
    except:
        dts = np.diff(result['timestamp'])
        out = dts[dts <= threshold].sum()
    return out

tick_variables = ['timestamp', 'qdot', 'action', 'movement_cum_dt', 'z_state', 'finished_reset', 'gripper', 'gripper_retval', 'gripper_state_at_ret', 'start_press', 'hover_state', 'xyz']
results = {}
try:
    for subject, subject_sessions in robot_random.sequence_start_end.items():
        results[subject] = []
        for folder, item in subject_sessions.items():
            for fname, keep_trials in item.items():
                print(subject, folder, fname, keep_trials)
                log = process_logs.process_logs(head / folder / 'robot_logs' / fname)
                if folder in ['exp_2024-03-18_12-59-19', 'exp_2024-05-01_12-21-58']:
                    resample_logs.resample_logs(log, robot_random.resample_start_end[folder][fname])
                for trial_index in keep_trials:
                    start, end = log['trial_start_idx'][trial_index], log['trial_end_idx'][trial_index]
                    tmp = {}

                    for key in tick_variables:
                        try:
                            tmp[key] = log[key][start:end].copy()
                        except:
                            print(f'skipping {key} for {folder}, {fname}')
                    tmp['trial_time'] = log['timestamp'][end-1] - log['timestamp'][start]
                    
                    try:
                        tmp['n_actions'] = (log['gripper_retval'][start:end] != -1).sum()
                    except:
                        # legacy for 2024-03-18
                        tmp['n_actions'] = np.isin(log['gripper'][start:end], [2, 3]).sum()
                    tmp['total_movement_time'] = get_total_movement_time(tmp)
                    
                    tmp['all_objs_list'], tmp['all_objs_ids'], tmp['all_objs_idx'] = log['all_objs_list'], log['all_objs_ids'], log['all_objs_idx']
                    tmp['start'] = start
                    tmp['end'] = end
                    
                    if (subject, folder, fname, trial_index) == robot_random.trajectory_info:
                        print('keeping trajectory for ', (subject, folder, fname, trial_index))
                    else:
                        for key in tick_variables:
                            try:
                                del tmp[key]
                            except:
                                pass
                    results[subject].append(tmp)
except Exception as e:
    print(e, traceback.format_exc())
    pass

S2 exp_2024-03-18_12-59-19 exp_2024-03-18_12-34-06.log [0]
skipping movement_cum_dt for exp_2024-03-18_12-59-19, exp_2024-03-18_12-34-06.log
skipping z_state for exp_2024-03-18_12-59-19, exp_2024-03-18_12-34-06.log
skipping finished_reset for exp_2024-03-18_12-59-19, exp_2024-03-18_12-34-06.log
skipping gripper_retval for exp_2024-03-18_12-59-19, exp_2024-03-18_12-34-06.log
skipping gripper_state_at_ret for exp_2024-03-18_12-59-19, exp_2024-03-18_12-34-06.log
skipping start_press for exp_2024-03-18_12-59-19, exp_2024-03-18_12-34-06.log
skipping hover_state for exp_2024-03-18_12-59-19, exp_2024-03-18_12-34-06.log
S2 exp_2024-03-18_12-59-19 exp_2024-03-18_12-44-36.log [0]
skipping movement_cum_dt for exp_2024-03-18_12-59-19, exp_2024-03-18_12-44-36.log
skipping z_state for exp_2024-03-18_12-59-19, exp_2024-03-18_12-44-36.log
skipping finished_reset for exp_2024-03-18_12-59-19, exp_2024-03-18_12-44-36.log
skipping gripper_retval for exp_2024-03-18_12-59-19, exp_2024-03-18_12-44-36.log
ski

In [7]:
with open('../../data/robot_compiled/sequences_compiled_stats.pickle', 'wb') as f:
    pickle.dump(results, f)